In [3]:
import torchvision
FMNIST_train = torchvision.datasets.FashionMNIST('./fmnist', train=True, download = True, transform = torchvision.transforms.ToTensor())
FMNIST_test = torchvision.datasets.FashionMNIST('./fmnist', train=False, download = False, transform = torchvision.transforms.ToTensor())

In [2]:
%load_ext tensorboard
%tensorboard --logdir logs/hparam_tuning

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 15050), started 0:00:02 ago. (Use '!kill 15050' to kill it.)

In [3]:
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from matplotlib import pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torchvision.transforms import ToTensor, PILToTensor, Compose
from torch.utils.data import Dataset, DataLoader

2024-04-02 09:12:40.161266: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 09:12:40.161307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 09:12:40.161313: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-02 09:12:40.165546: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Сформируем стандартные функции для обучения по батчу, по эпохам и обертку для инициализации оптимизатора и датасета для обучения

In [4]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_function):
    model.train()
    optimizer.zero_grad()
    output = model(x_batch.to(model.device()))
    loss = loss_function(output, y_batch.to(model.device()))
    loss.backward()
    optimizer.step()
    return loss.cpu().item()

In [5]:

def train_epoch(train_generator, model, loss_function, optimizer, callback = None):
    epoch_loss = 0
    total = 0
    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        batch_loss = train_on_batch(model, batch_of_x, batch_of_y, optimizer, loss_function)
        
        if callback is not None:
            with torch.no_grad():
                callback(model, batch_loss)
            
        epoch_loss += batch_loss*len(batch_of_x)
        total += len(batch_of_x)
    
    return epoch_loss/total

In [6]:
def trainer(count_of_epoch, 
            batch_size, 
            dataset,
            model, 
            loss_function,
            optimizer,
            lr = 0.001,
            callback = None):

    optima = optimizer(model.parameters(), lr=lr)

    iterations = tqdm.tqdm(range(count_of_epoch), desc='epoch')
    iterations.set_postfix({'train epoch loss': np.nan})
    epoch_loss = None
    for it in iterations:
        batch_generator = tqdm.tqdm(
            torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True), 
            leave=False, total=len(dataset)//batch_size+(len(dataset)%batch_size> 0))
        
        epoch_loss = train_epoch(train_generator=batch_generator, 
                    model=model, 
                    loss_function=loss_function, 
                    optimizer=optima, 
                    callback=callback)
        
        iterations.set_postfix({'train epoch loss': epoch_loss})
    return epoch_loss

Блоки энкодера CNN (DownBlock) сформируем на основе двух сверточных слоев, каждый из которых состоит из собственно свертки, потом нормализации по батчу ,если используется, и функции активации. Также после функции активации для кадого слоя кроме последнего, стоит Dropout с заданным параметром. Полсе последнего слоя свертки стоит пуллинг. Т.к. по заданию нам требуется менять струкутру сети, то для удобства каждый сверточный слой формируем с помощью nn.Sequential, что позволит добавлять в него модули по требованию. Для простоты определения параметров используем паддинг.

In [7]:
class DownBlock(nn.Module):
    def __init__(self, in_channel, out_channel, down=True, dropout = 0.5, kernel_size = 3, batch_norm = True, pooling ='maxpool'):
        
        super().__init__()
        self.conv1 = nn.Sequential()
        self.conv1.add_module('convl1',nn.Conv2d(in_channel, out_channel, kernel_size, padding='same', padding_mode = 'reflect')),
        if batch_norm:
            self.conv1.add_module('batch1',nn.BatchNorm2d(out_channel)),
        self.conv1.add_module('relu1',nn.ReLU()),
        self.conv1.add_module('dropout1',nn.Dropout(dropout))

        self.conv2 = nn.Sequential()
        self.conv2.add_module('convl2',nn.Conv2d(out_channel, out_channel, kernel_size, padding='same', padding_mode = 'reflect')),
        if batch_norm:
            self.conv2.add_module('batch2', nn.BatchNorm2d(out_channel)),
        self.conv2.add_module('relu2', nn.ReLU())

        if pooling == 'maxpool':
            self.pooling = torch.nn.MaxPool2d(2)
        elif 'avgpool':
            self.pooling = torch.nn.AvgPool2d(2)
        
        # TODO

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pooling(x)
        return x

Классификацию мы проводим на ветораях эмбедингов полученных от энкодера, у нас стоит задача классификации, а не восстановления исходных объектов. Поэтому слои декодера определять не имеет смысла.

Определеим общую структуру нашей сети. Первые два сверточных слоя сильно отличаются по структуре от пердыдущих, поэтому их придется определить вручную. Параметры остальных слоев получаются из предыдущих делением на 2, т.к. мы использовали паддинг до нужных размеров. Выход от сверточных слоев мы подаем на вход полносвязанной сети как веторы, которая используется как классификационная сеть. На выходе полносвяанной сети соотвественно получаем вероятности каждого класса.

In [9]:
class CNN(nn.Module):
    def device(self):
        return next(self.parameters()).device
    def __init__(self,
                 num_classes, 
                 min_channels=32,
                 max_channels=512,
                 colors = 1,
                 num_down_blocks=4, image_size = 32, kernel_size = 3, dropout = 0.2, batch_norm = True, pooling ='maxpool'):
        super(CNN, self).__init__()
        self.num_classes = num_classes
        self.num_down_blocks = num_down_blocks
        self.conv1 = nn.Sequential()
        self.conv1.add_module('convl1', nn.Conv2d(colors, min_channels, kernel_size, padding='same', padding_mode = 'reflect')),
        if batch_norm:
            self.conv1.add_module('batch1',nn.BatchNorm2d(min_channels)),
        self.conv1.add_module('relu1', nn.ReLU()),
        self.conv1.add_module('dropout1', nn.Dropout(dropout))

        self.conv2 = nn.Sequential()
        self.conv2.add_module('conv2', nn.Conv2d(min_channels, min_channels, kernel_size, padding='same', padding_mode = 'reflect')),
        if batch_norm:
            self.conv2.add_module('batch2', nn.BatchNorm2d(min_channels)),
        self.conv2.add_module('relu2', nn.ReLU())

        
        self.down_blocks = nn.ModuleList()
        prev_chanel = min_channels
        self.chanel_size = []
        for i in range(self.num_down_blocks):
            self.chanel_size.append(prev_chanel)
            out_chanel = prev_chanel * 2
            self.down_blocks.append(DownBlock(prev_chanel, out_chanel, dropout=dropout, kernel_size = 3, batch_norm = True, pooling ='maxpool'))
            prev_chanel = out_chanel
        #print((image_size + (2 << (num_down_blocks - 1)) - 1))
        multiplier =  ((image_size + (2 << (num_down_blocks - 1)) - 1)>> num_down_blocks)
        #print(multiplier)
        if multiplier > 0:
            prev_chanel *= multiplier**2
        #print(prev_chanel)
        self.flatten = torch.nn.Flatten()
        self.linerlayer1 = nn.Sequential(
            nn.Linear(prev_chanel, int(prev_chanel / 4)), 
            nn.ReLU()
        )
        self.linerlayer2 = nn.Sequential(
            nn.Linear(int(prev_chanel / 4), int(prev_chanel / 8)), 
            nn.ReLU()
        )
        self.linerlayer3 = nn.Sequential(
            nn.Linear(int(prev_chanel / 8), num_classes)
        )
        self.tensordevice = nn.Parameter()

    def forward(self, inputs):
        # TODO
        width = inputs.shape[-1]
        height = inputs.shape[-2]
        width_ = 2 ** self.num_down_blocks - ((width - 1) % (2 ** self.num_down_blocks) + 1)
        height_ = 2 ** self.num_down_blocks- ((height- 1) % (2 ** self.num_down_blocks)  + 1)
        padding = torch.nn.ReflectionPad2d((0, width_, 0, height_))
        x = padding(inputs)
        x = self.conv1(x)
        x = self.conv2(x)
        x_skip = [x]
        for i in range(self.num_down_blocks):
            x = self.down_blocks[i](x)
        #print(x.shape)
        x = self.flatten(x)
        x = self.linerlayer1(x)
        x = self.linerlayer2(x)
        x = self.linerlayer3(x)
        logits = x
        #assert logits.shape == (inputs.shape[0], self.num_classes, inputs.shape[2], inputs.shape[3]), 'Wrong shape of the logits'
        return logits

In [10]:
from torchvision import datasets, transforms

In [11]:
train_dataloader = torch.utils.data.DataLoader(dataset=FMNIST_train, batch_size=64, shuffle=True)

In [12]:
cnn = CNN(10, min_channels=4, image_size = 28, colors = 1, num_down_blocks=2)

В качестве функции потерь для задачи классификации возьмем кросс-энтропию, в качестве оптимизатора Adam без регуляризации (weight decay).

In [13]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

Обучаем на GPU

In [14]:
cnn.cuda()

CNN(
  (conv1): Sequential(
    (convl1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=same, padding_mode=reflect)
    (batch1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (dropout1): Dropout(p=0.2, inplace=False)
  )
  (conv2): Sequential(
    (conv2): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=same, padding_mode=reflect)
    (batch2): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU()
  )
  (down_blocks): ModuleList(
    (0): DownBlock(
      (conv1): Sequential(
        (convl1): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=same, padding_mode=reflect)
        (batch1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
      )
      (conv2): Sequential(
        (convl2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=same, padding_mo

In [15]:
cnn.device()

device(type='cuda', index=0)

In [16]:
"""trainer(20, 
        64, 
        FMNIST_train,
        cnn, 
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam,
        lr = 0.001
        )"""

'trainer(20, \n        64, \n        FMNIST_train,\n        cnn, \n        torch.nn.CrossEntropyLoss(),\n        torch.optim.Adam,\n        lr = 0.001\n        )'

Для получения логов используем tensorboard

In [17]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

Для логгирования всех метрик на этапе обучения лучше всего использщовать отдельный интерфейс. В нем мы каждый шаг записываем значения loss на трейне. И раз в несколько сотен шаг записывем значение метрик на валидационной выборке. Что позволяет отслеживать темп обучения и его качетсво, при этом не сильно теряя во времени обучения 

In [18]:
class callback():
    def __init__(self, writer, dataset, loss_function, metrics, hparams,  delimeter = 100, batch_size=64):
        self.step = 0
        self.writer = writer
        self.delimeter = delimeter
        self.loss_function = loss_function
        self.batch_size = batch_size
        self.hparams = hparams
        self.dataset = dataset
        self.metrics = metrics

    def forward(self, model, loss):
        self.step += 1
        with self.writer.as_default():
            hp.hparams(self.hparams)
            tf.summary.scalar(self.metrics + '/train', loss, self.step)
        
        if self.step % self.delimeter == 0:
            
            batch_generator = torch.utils.data.DataLoader(dataset = self.dataset, 
                                                          batch_size=self.batch_size)
            
            pred = []
            real = []
            test_loss = 0
            model.eval()
            for it, (x_batch, y_batch) in enumerate(batch_generator):
                x_batch = x_batch.to(model.device())

                output = model(x_batch)

                test_loss += self.loss_function(output, y_batch.to(model.device())).cpu().item()*len(x_batch)

                pred.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
            
            test_loss /= len(self.dataset)
            with self.writer.as_default():
                tf.summary.scalar(self.metrics + '/test', test_loss, self.step)

            x = x_batch[-10:]
          
    def __call__(self, model, loss):
        return self.forward(model, loss)

В качестве алогритма по перебору гиперпараметров больше всего приглянулась optuna

In [19]:
import optuna
import optuna_dashboard

Определим сетку по которй будем перебирать гиперпараметры, по заданию не требуется перебирать все варианты, поэтому построим набольшую сетку

In [20]:
HP_KERNEL_SIZE = hp.HParam('num_units', hp.IntInterval(2, 5))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.3, 0.5))
HP_POOLING = hp.HParam('poolling', hp.Discrete(['maxpool', 'avgpool']))
HP_BATCH_NORM = hp.HParam('batch_norm', hp.Discrete([True, False]))
HP_LAYERS_COUNT = hp.HParam('layers_count', hp.IntInterval(2, 5))
METRICS_NAME = "LOSS"
DELIMETER = 100
IMAGE_SIZE = 28
EPOCHS = 10
BATCH_SIZE  = 128
COLORS = 1
MIN_CHANNELS = 4

Это нужно для логгирования каждого значения гиперпараметра в tf

In [30]:
logs_dir = 'logs/hparam_tuning'
writer_hparam = tf.summary.create_file_writer('logs/hparam_tuning')
with writer_hparam.as_default():
  hp.hparams_config(
    hparams=[HP_KERNEL_SIZE, HP_DROPOUT, HP_POOLING, HP_BATCH_NORM, HP_LAYERS_COUNT],
    metrics=[hp.Metric(METRICS_NAME  + '/train', display_name='Loss')],
  )

Наконец запустим наше обучение. На вход optuna нужно передать функцию, которая по интерфейсу trial (соответсвует одной попытке, из которой можно получить значение гиперпараметров предлагаемых optnua) выдает значение оптимизируемой метрики. На выходе она выдаетрезультаты лучшей попытки попытки, оптимальные значение гиперпараметров. А также позволяет визуализировать значение гиперпараметров.

In [31]:

def objective(trial):
    run_number = trial.number
    run_name = logs_dir + f'/run-{run_number}'
    print("Current run: " + run_name)
    kernel_size =  trial.suggest_int('num_units', 2, 5)
    dropout = trial.suggest_float('dropout', 0.3, 0.5)
    pooling = trial.suggest_categorical('poolling', ['maxpool', 'avgpool'])
    batch_norm = trial.suggest_categorical('batch_norm', [True, False])
    layers_count = trial.suggest_int('layers_count', 2, 5)
    writer = tf.summary.create_file_writer(run_name)
    cnn = CNN(10, min_channels=MIN_CHANNELS , colors = COLORS , image_size=IMAGE_SIZE,  num_down_blocks=layers_count, kernel_size=kernel_size, dropout=dropout, pooling=pooling, batch_norm=batch_norm)
    cnn.cuda()
    hparams = {
      HP_KERNEL_SIZE: kernel_size,
      HP_DROPOUT: dropout,
      HP_POOLING:  pooling,
      HP_BATCH_NORM: batch_norm,
      HP_LAYERS_COUNT: layers_count,
    }
    loss = trainer(EPOCHS, 
        BATCH_SIZE, 
        FMNIST_train,
        cnn, 
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam,
        lr = 0.001,
        callback = callback(writer, FMNIST_train, torch.nn.CrossEntropyLoss(), METRICS_NAME , hparams, DELIMETER))
    print(loss)
    return loss
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2024-04-02 09:44:46,505] A new study created in memory with name: no-name-3dac8b69-f95d-4b39-9ec8-66a1d3e0721a


Current run: logs/hparam_tuning/run-0


epoch: 100%|██████████| 10/10 [05:17<00:00, 31.75s/it, train epoch loss=0.252]
[I 2024-04-02 09:50:04,014] Trial 0 finished with value: 0.25240968391100566 and parameters: {'num_units': 4, 'dropout': 0.4019725961076505, 'poolling': 'maxpool', 'batch_norm': True, 'layers_count': 2}. Best is trial 0 with value: 0.25240968391100566.


0.25240968391100566
Current run: logs/hparam_tuning/run-1


epoch: 100%|██████████| 10/10 [06:15<00:00, 37.55s/it, train epoch loss=0.347]
[I 2024-04-02 09:56:19,495] Trial 1 finished with value: 0.34670891030629475 and parameters: {'num_units': 4, 'dropout': 0.4040352602677366, 'poolling': 'avgpool', 'batch_norm': False, 'layers_count': 5}. Best is trial 1 with value: 0.34670891030629475.


...

0.34661933118502297
Current run: logs/hparam_tuning/run-8


epoch: 100%|██████████| 10/10 [06:09<00:00, 36.97s/it, train epoch loss=0.316]
[I 2024-04-02 10:39:22,117] Trial 8 finished with value: 0.31561945128440855 and parameters: {'num_units': 3, 'dropout': 0.3522595966607735, 'poolling': 'maxpool', 'batch_norm': False, 'layers_count': 5}. Best is trial 3 with value: 0.35567838447888694.


0.31561945128440855
Current run: logs/hparam_tuning/run-9


epoch: 100%|██████████| 10/10 [06:23<00:00, 38.37s/it, train epoch loss=0.349]
[I 2024-04-02 10:45:45,818] Trial 9 finished with value: 0.3490602419694265 and parameters: {'num_units': 3, 'dropout': 0.4435010060686884, 'poolling': 'avgpool', 'batch_norm': True, 'layers_count': 5}. Best is trial 3 with value: 0.35567838447888694.


0.3490602419694265


## Вывод
По результатам переборов гиперпараметров maxpool показывает себя лучше, чем avgpool. Что объяснентся тем, что maxpool делает свертку инвариантной относительно расположения интересующей части картинки. Т.е. для maxpool по потсроению не важно в какой части изображения находится классифицируемый объект. Ожидаемо нормализация также улучшила качество классификации. Также благоприятно сказывется на обучении увеличение кол-во слоев. А вот зависимоти от dropout не выявилось, скорее всего оптимальные значения лежат в пределе исследуемых.

В целом результаты получились ожидаемые пуллинг по максимуму оказался лучше, чем усредняющий, т.к. выделяет фичи из интересующей части изображения, а не усредняет их по всем. Разумное увеличение количества слоев таже увеличвет качество определяемых фичей и соотвественно качество классификации. 